# Notities

## Handige links
* [Statline portaal](https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS)
* [Snelstartgids open data](https://www.cbs.nl/nl-nl/onze-diensten/open-data/statline-als-open-data/snelstartgids)
* [repo Github voor cbsodata](https://github.com/J535D165/cbsodata)
* [PyPi documentatis cbsodata](https://pypi.org/project/cbsodata/)
* [Sphinx docs cbsodata](https://cbsodata.readthedocs.io/en/latest/index.html)
* [GitHub voorbeelden OData 3](https://github.com/statistiekcbs/CBS-Open-Data-v3)
* [GitHub voorbeelden OData 4](https://github.com/statistiekcbs/CBS-Open-Data-v4/tree/master/Python)
* [Statline voor derden](https://www.cbs.nl/nl-nl/cijfers/statline/statline-voor-derden)

# Imports

In [1]:
import pandas as pd
import cbsodata

# Functies

In [22]:
def get_relevant_tables(url:str, keywords=[], select_columns=['Identifier', 'ShortTitle','ColumnCount', 'RecordCount', 'Updated', 'Period', 
           'Summary', 'Frequency', 'ShortDescription', 'ExplanatoryText']):
    '''
    
    '''
    tables_list = cbsodata.get_table_list(catalog_url=url)
    df_tables = pd.DataFrame(tables_list)
    print(f"Total number of tables in this url is: {df_tables.shape[0]}")
    print(f"The columns in these tables are: {df_tables.columns}")
    total_identifier_list = []
    for keyword in keywords:
        identifier_list = [i['Identifier'] for i in tables_list if keyword.lower() in i['Title'].lower()]
        total_identifier_list = total_identifier_list + identifier_list
    df = df_tables[df_tables['Identifier'].isin(total_identifier_list)][select_columns]
    return df

# Explore possible tables

In [2]:
toc = pd.DataFrame(cbsodata.get_table_list())

In [16]:
toc.shape[0]

4881

## URL = 'opendata.cbs.nl'

In [30]:
df_tables_cbs_opendata = get_relevant_tables(url='opendata.cbs.nl', 
                                             keywords=['wmo'])
df_tables_cbs_opendata

Total number of tables in this url is: 4881
The columns in these tables are: Index(['ApiUrl', 'Catalog', 'ColumnCount', 'DefaultPresentation',
       'DefaultSelection', 'ExplanatoryText', 'FeedUrl', 'Frequency',
       'GraphTypes', 'ID', 'Identifier', 'Language', 'MetaDataModified',
       'Modified', 'OutputStatus', 'Period', 'ReasonDelivery', 'RecordCount',
       'SearchPriority', 'ShortDescription', 'ShortTitle', 'Source', 'Summary',
       'SummaryAndLinks', 'Title', 'Updated'],
      dtype='object')


,Identifier,ShortTitle,ColumnCount,RecordCount,Updated,Period,Summary,Frequency,ShortDescription,ExplanatoryText
645,84907NED,"Wmo-cliënten; financiering, wijken, 2020",9,14128,2020-11-26T02:00:00,2020,Wmo-cliënten met maatwerkarrangement; financie...,Perhalfjaar,\nIn deze tabel wordt het aantal cliënten met...,
646,84664NED,"Wmo-cliënten; financiering, wijken, 2019",9,42228,2020-11-30T02:00:00,2019,Wmo-cliënten met maatwerkarrangement; financie...,Stopgezet,\nIn deze tabel wordt het aantal cliënten met...,
647,84421NED,"Wmo-cliënten; financiering, wijken, 2018",9,41592,2019-12-20T02:00:00,2018,Wmo-cliënten met maatwerkarrangement; financie...,Stopgezet,\nIn deze tabel wordt het aantal cliënten met...,
648,83818NED,"Wmo-cliënten; financiering, wijken, 2017",9,41496,2018-12-12T02:00:00,2017,Wmo-cliënten met maatwerkarrangement; financie...,Stopgezet,\nIn deze tabel wordt het aantal cliënten met...,
649,83620NED,"Wmo-cliënten; financiering, wijken 2016",9,40452,2018-07-06T02:00:00,2016,Wmo-cliënten met maatwerkarrangement; financie...,Stopgezet,\nIn deze tabel wordt het aantal cliënten met...,
650,83267NED,"Wmo-cliënten; financiering, wijken 2015",9,38508,2017-07-04T02:00:00,1e halfjaar 2015 - 2e halfjaar 2015,Wmo-cliënten met maatwerkarrangement; financie...,Stopgezet,\nIn deze tabel wordt het aantal cliënten met...,
651,84908NED,"Wmo_cliënten; type, wijken, 2020",9,21192,2020-11-26T02:00:00,2020,"Wmo-cliënten, maatwerkarrangementen (hulp in h...",Perhalfjaar,\nIn deze tabel wordt het aantal cliënten met...,
652,84753NED,"Wmo-cliënten; type maatwerk, wijken 2019",9,63342,2020-11-30T02:00:00,2019,"Wmo-cliënten, maatwerkarrangementen (hulp in h...",Stopgezet,\nIn deze tabel wordt het aantal cliënten met...,
653,84752NED,"Wmo-cliënten; type maatwerk, wijken 2018",9,62388,2020-05-13T02:00:00,2018,"Wmo-cliënten, maatwerkarrangementen (hulp in h...",Stopgezet,\nIn deze tabel wordt het aantal cliënten met...,
654,84751NED,"Wmo-cliënten; type maatwerk, wijken 2017",9,62244,2020-05-13T02:00:00,2017,"Wmo-cliënten, maatwerkarrangementen (hulp in h...",Stopgezet,\nIn deze tabel wordt het aantal cliënten met...,


## URL = 'dataderden.cbs.nl'

In [29]:
df_tables_cbs_derden = get_relevant_tables(url='dataderden.cbs.nl', 
                                             keywords=['wmo'])
df_tables_cbs_derden

Total number of tables in this url is: 499
The columns in these tables are: Index(['ApiUrl', 'Catalog', 'ColumnCount', 'DefaultPresentation',
       'DefaultSelection', 'ExplanatoryText', 'FeedUrl', 'Frequency',
       'GraphTypes', 'ID', 'Identifier', 'Language', 'MetaDataModified',
       'Modified', 'OutputStatus', 'Period', 'ReasonDelivery', 'RecordCount',
       'SearchPriority', 'ShortDescription', 'ShortTitle', 'Source', 'Summary',
       'SummaryAndLinks', 'Title', 'Updated'],
      dtype='object')


,Identifier,ShortTitle,ColumnCount,RecordCount,Updated,Period,Summary,Frequency,ShortDescription,ExplanatoryText
84,40072NED,Gefactureerde eigen bijdrage Wmo thuis,6,720,2020-11-26T02:00:00,2015-2019,"Gemiddelde gefactureerde eigen bijdrage, aanta...",Perjaar,\nDeze tabel toont de gemiddelde gefactureerde...,
85,40073NED,Gefactureer. eigen bijdrage Wmo verblijf,6,720,2020-11-26T02:00:00,2015-2019,"Gemiddelde gefactureerde eigen bijdrage, aanta...",Perjaar,\nDeze tabel toont de gemiddelde gefactureerde...,
87,40041NED,Opgelegde eigen bijdrage,5,31,2020-11-27T02:00:00,2013 januari-maart - 2020 januari-september,"AWBZ en Wmo tot 2015, vanaf 2015 Wlz en Wmo\no...",Perkwartaal,\nDeze tabel bevat cumulatieve kwartaalcijfers...,
95,40061NED,Gebruik Wmo-maatwerkvoorzieningen,7,2361600,2020-11-30T02:00:00,2015-2020,Aantal cliënten met een maatwerkvoorziening in...,Perhalfjaar,\nIn deze tabel wordt het aantal cliënten met ...,
98,40060NED,Gebruik bijdrageplichtig Wmo-maatwerk,9,8302500,2020-11-04T02:00:00,2015-2019,"Personen met zorggebruik, geslacht, leeftijd, ...",Perjaar,\nDeze tabel toont het aantal en aandeel perso...,
131,40039NED,"Wmo; baten/lasten, gemeente, 2009-2014",15,8154,2016-08-05T02:00:00,2009-2014,"Wmo, baten en lasten, gemeente, regio, gemeent...",Stopgezet,\nDeze tabel gaat over de baten en lasten per ...,
132,40038NED,"Wmo; baten en lasten, regio, 2009-2014",15,1170,2016-08-05T02:00:00,2009-2014,"Wmo, baten en lasten, gemeente, regio, gemeent...",Stopgezet,\nDeze tabel gaat over de baten en lasten per ...,
133,40050NED,"Wmo/Jeugdwet; baten en lasten, regio",16,1560,2021-01-11T02:00:00,2017-2020,"Wet maatschappelijke ondersteuning, Jeugdwet, ...",Perjaar,\nDeze tabel gaat over de baten en lasten per ...,
134,40043NED,"Wmo/Jeugdwet; baten, lasten, 2015-2016",12,780,2018-05-15T02:00:00,2015-2016,"Wmo/Jeugdwet, baten en lasten per functie (bel...",Stopgezet,\nDeze tabel gaat over de baten en lasten per ...,


In [28]:
# DRAAIT LANG (+/- 10 min)
dict_cbs_wmo_tables = {}
for i in cbs_wmo_identifiers:
    print(i)
    try:
        df = pd.DataFrame(cbsodata.get_data(str(i), catalog_url='dataderden.cbs.nl'))
    except Exception:
        df = pd.DataFrame()
        pass
    dict_cbs_wmo_tables[i] = df

40072NED
40073NED
40041NED
40061NED
40060NED
40039NED
40038NED
40050NED
40043NED
